In [1]:
import pandas as pd
df = pd.read_csv ('medium_data.csv')
# https://www.kaggle.com/datasets/dorianlazar/medium-articles-dataset

df.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [2]:

# short_articles = df[(df['reading_time'] == 5) & (df['publication'] == "Towards Data Science")]
# short_articles.shape


reading_times = sorted(df['reading_time'].unique())
publications = list(df['publication'].unique())
print("reading_times:", reading_times)
print("publications:", publications)
print()

for t in reading_times:
    for p in publications:
        articles = df[(df['reading_time'] == t) & (df['publication'] == p)]
        shape = articles.shape
        if shape[0] != 0:
            print(t, p, ' '*(24-len(p)), shape[0])
    print()
    
    

reading_times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 31, 32, 33, 36, 40, 55]
publications: ['Towards Data Science', 'UX Collective', 'The Startup', 'The Writing Cooperative', 'Data Driven Investor', 'Better Marketing', 'Better Humans']

0 The Startup               2

1 Towards Data Science      3
1 UX Collective             1
1 The Startup               7
1 The Writing Cooperative   1
1 Data Driven Investor      2

2 Towards Data Science      22
2 UX Collective             19
2 The Startup               72
2 The Writing Cooperative   12
2 Data Driven Investor      40
2 Better Marketing          2
2 Better Humans             1

3 Towards Data Science      87
3 UX Collective             75
3 The Startup               340
3 The Writing Cooperative   87
3 Data Driven Investor      141
3 Better Marketing          17
3 Better Humans             2

4 Towards Data Science      162
4 UX Collective             86
4 The Startup     

In [3]:
subset_df = df[(df['reading_time'] == 5) & (df['publication'] == 'The Startup')]
subset_df.head()
subset_df.shape

(649, 10)

In [4]:
# pip install -U sentence-transformers
# https://www.sbert.net/docs/quickstart.html

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-MiniLM-L6-v2')

# #Our sentences we like to encode
# sentences = ['This framework generates embeddings for each input sentence',
#     'Sentences are passed as a list of string.', 
#     'The quick brown fox jumps over the lazy dog.']

# #Sentences are encoded by calling model.encode()
# sentence_embeddings = model.encode(sentences)

# #Print the embeddings
# for sentence, embedding in zip(sentences, sentence_embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")


In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

#Sentences are encoded by calling model.encode()
emb1 = model.encode("This is a red cat with a hat.")
emb2 = model.encode("Have you seen my red cat?")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim) # cos_sim is of type: <class 'torch.Tensor'>
print("in float form:", float(cos_sim))

Cosine-Similarity: tensor([[0.6153]])
in float form: 0.6153064966201782


In [9]:
df['full_title'] = df.apply(lambda row: row['title'] + ' ' + row['subtitle']
                          if not pd.isna(row['subtitle']) else row['title'], axis=1)

In [10]:
subset_df = subset_df.reset_index(drop=True)
df = subset_df # just so I don't have to replace too much code when we do the whole thing

In [11]:
df_rows = len(df['full_title'])
distances = [["NULL" for _ in range(df_rows)] for _ in range(df_rows)]

In [12]:
k = 5 # or whatever

nearest_neighbor_data = [[float('inf') for _ in range(k)] for _ in range(df_rows)]

yes


In [29]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

for i in range(df_rows):
    for j in range(i + 1, df_rows): # i + 1 so it doesn't compute the distance to itself
        emb1 = model.encode(df['full_title'][i])
        emb2 = model.encode(df['full_title'][j])
        cos_sim = float(util.cos_sim(emb1, emb2))
        if cos_sim < 0:
            cos_sim = -cos_sim
        distances[i][j] = cos_sim
        
        # check if cos_sim is within the first k smallest values
        # if so, store in proper place
        for x in range(len(nearest_neighbor_data[i])):
            val = nearest_neighbor_data[i][x]
            if cos_sim < val:
                nearest_neighbor_data[i][x] = cos_sim
        
                
        # TODO move other mins down to make room
        # probably make prev loop recursive
        
        if j == 50:
            break
    break